In [1]:
import torch

# from speech2spikes import S2S

import torchvision
from torch.utils.data import DataLoader, ConcatDataset
import torchaudio

from torch_mate.data.utils import FewShot
from torch_mate.utils import get_device

from neurobench.data.datasets.MSWC import MSWC
from neurobench.examples.few_shot_learning.utils import train_using_MAML
from neurobench.models import M5
from neurobench.utils import Dict2Class

from neurobench.benchmarks import Benchmark

In [2]:
ROOT = "//scratch/p306982/data/fscil/mswc/"

In [3]:
train = lambda net, data: net

model = M5(n_input=48, n_output=40)
eval_set = MSWC(root=ROOT, subset="evaluation")

In [6]:
fscil_set = FewShot(eval_dataset, 10, 5, 100, None, (100, 100), True, True, None, 200, torch.nn.Identity(), None)

In [7]:
few_shot_dataset = FewShot(eval_set, 10, 5, 100,
                            incremental=True,
                            cumulative=True,
                            support_query_split=(100,100),
                            samples_per_class=200,
                            transform=torch.nn.Identity())

In [ ]:
base_test_set = MSWC(root=ROOT, subset="base", procedure="testing")
base_test_data_loader = DataLoader(base_test_set, batch_size=32, prefetch_factor=3, num_workers=4)

benchmark = Benchmark(model, metric_list=[[],["classification_accuracy"]], dataloader=None)

In [14]:
for session, (X, y) in enumerate(few_shot_dataset):
    print(f"Session: {session}")

    X_train, X_test = X
    y_train, y_test = y

    X_train = X_train.squeeze()
    y_train = y_train.squeeze()
    X_test = X_test.squeeze()
    y_test = y_test.squeeze()


    model = train(model, (X_train[0], y_train[0]))


    session_query_set = torch.utils.data.TensorDataset(X_test,y_test)
    


    ### For Benchmark session testing
    # 1. Create dataloader from X_test, y_test
    session_test_dataloader = SOMETHING(X_test, y_test)
    # 2. Zip this dataloader with base_test_dataloader
    full_session_test = zip(base_test_data_loader, session_test_dataloader)
    # 3. Feed as data to Benchmark
    session_results = benchmark.run(data = full_session_test)

    # print(session_results)

KeyboardInterrupt: 